> docker run python:2 python -c "import platform; print 'Python running on arch: %s' %platform.machine()"
Python running on arch: x86_64
The following code shows another example using Ubuntu Linux and printing the architecture using uname:


 > docker run ubuntu uname -a
Linux 436761beed66 4.9.125-linuxkit #1 SMP Fri Sep 7 08:20:28 UTC 2018 x86_64 x86_64 x86_64 GNU/Linux
You might want to deploy your application on an Arm server on AWS, or on an Arm embedded board. Developing an application using Docker on an x86 machine may introduce incompatibilities when running the same software on an Arm machine. Docker Desktop allows you to create and test Arm images from your Windows desktop.

The best way to create images for Arm is to use the new buildx command which is included in Docker Desktop. The command usage is shown in this code:


 > docker buildx

Usage:  docker buildx COMMAND

Build with BuildKit

Management Commands:
  imagetools  Commands to work on images in registry

Commands:
  bake        Build from a file
  build       Start a build
  create      Create a new builder instance
  inspect     Inspect current builder instance
  ls          List builder instances
  rm          Remove a builder instance
  stop        Stop builder instance
  use         Set the current builder instance

Run 'docker buildx COMMAND --help' for more information on a command.
Let’s use a simple Dockerfile to see how the same Dockerfile supports multiple architectures with no modifications. Use a text editor to create a two-line Dockerfile with the contents that are shown in the following code:


FROM alpine
RUN apk --no-cache add curl
The –no-cache argument is a useful trick to not cache the index and keep containers small.

Now we look at how to build images for different platforms using buildx. Follow these steps:

Create a new builder instance and use it, then build three images which are stored locally, this can be seen in the following code:


 > docker buildx create --name mybuilder
> docker buildx use mybuilder
> docker buildx build --platform linux/amd64 -t alpine-amd64 --load .
> docker buildx build --platform linux/arm64 -t alpine-arm64 --load .
> docker buildx build --platform linux/arm/v7 -t alpine-arm32 --load .
Run each image, including the Arm images, on the local desktop, as you can see in the following code:


 > docker run alpine-amd64 uname -a
Linux 4bc3bd4b8ff0 4.9.125-linuxkit #1 SMP Fri Sep 7 08:20:28 UTC 2018 x86_64 Linux

> docker run alpine-arm64 uname -a
Linux 404631ac3379 4.9.125-linuxkit #1 SMP Fri Sep 7 08:20:28 UTC 2018 aarch64 Linux

> docker run alpine-arm32 uname -a
Linux 5a869d794098 4.9.125-linuxkit #1 SMP Fri Sep 7 08:20:28 UTC 2018 armv7l Linux
Now the images for each architecture are on this local machine. However, the goal is to save a single image that works on all platforms. This can be done with a single buildx command which pushes directly to a repository like Docker Hub. Using a repository is an easy way to migrate images to other machines, because the images can be pulled directly from the repository using any machine.

Create a Docker Hub account if you do not have one. You can go to hub.docker.com and click Sign Up.

Once pushed, buildx can be used to inspect the image and see that it supports three platforms, using the following code:


 > docker buildx build  --platform linux/amd64,linux/arm64,linux/arm/v7 -t jasonrandrews/alpine-test --push .

> docker buildx imagetools inspect jasonrandrews/alpine-test
Name:      docker.io/jasonrandrews/alpine-test:latest
MediaType: application/vnd.docker.distribution.manifest.list.v2+json
Digest:    sha256:6f36d248c3b139dc998cd7129a768cf068dd6371ecd6f027ce43c49b6bf80aa4

Manifests:
  Name:      docker.io/jasonrandrews/alpine-test:latest@sha256:aaf426c683e2b1369fdba62e6c420980402fc3706c59ec59eacf0d1ab419e719
  MediaType: application/vnd.docker.distribution.manifest.v2+json
  Platform:  linux/amd64

  Name:      docker.io/jasonrandrews/alpine-test:latest@sha256:5966f7b12b7c7ba3146102cf3079e57cccaf1de7228651661276dd1606d84108
  MediaType: application/vnd.docker.distribution.manifest.v2+json
  Platform:  linux/arm64

  Name:      docker.io/jasonrandrews/alpine-test:latest@sha256:12e131c1e16083f5d8a8dcaf8b52b10e78612cea439e98b1cd32fe9a60a3cefa
  MediaType: application/vnd.docker.distribution.manifest.v2+json
  Platform:  linux/arm/v7